# Large Language Model Prompt Engineering

***
In this lab, we will cover what is prompt engineering, what are some real world use cases, and advanced prompt engineering insights.
***

1. [Set Up](#1.-Set-Up)
2. [N-shots Prompting Concepts](#2.-N-shots-Prompting-Concepts)
3. [Chain of Thought (CoT) Prompting](#3.-Chain-of-Thought-(CoT)-Prompting)
4. [Tree of Thoughts (ToT) Prompting](#4.-Tree-of-Thoughts-(ToT)-Prompting)

## 1. Set Up

The endpoint URL is pointing to a AI21 Jurassic 2 Grande model deployed on Amazon Sagemaker endpoint. This will be the model used for this lab.

In [1]:
# !pip install boto3
# !pip install logging
!pip install requests

In [48]:
import logging
import json

import boto3
import requests

region = boto3.Session().region_name

logger = logging.getLogger('sagemaker')
logger.handlers.clear()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

url = "https://e165qoav65.execute-api.us-east-1.amazonaws.com/test/"
jurassic_url = "https://e165qoav65.execute-api.us-east-1.amazonaws.com/test/"
falcon_url = ""

In [380]:
def query_endpoint_with_json_payload_on_jurassic_llm(payload, url=jurassic_url):
    """Invoke a REST API via HTTP(S) request to interact with the model - returning the output text
    """
    response = requests.post(
        url,
        json=payload,
    )

    response_body = response.content

    response_json = json.loads(response_body.decode('utf-8'))
    if "ErrorCode" in response_json:
        msg = response_json.get("Message", "(No error details available)")
        raise RuntimeError(response_json["ErrorCode"] + ": " + msg)

    generated_text = response_json['completions'][0]['data']['text']
    return generated_text

def query_endpoint_with_json_payload_on_falcon_llm(payload, url=falcon_url):
    """Invoke a REST API via HTTP(S) request to interact with the model - returning the output text
   """

    response = requests.post(
        url,
        json=payload,
    )
    response_body = response.content

    response_json = json.loads(response_body.decode('utf-8'))
    if "ErrorCode" in response_json:
        msg = response_json.get("Message", "(No error details available)")
        raise RuntimeError(response_json["ErrorCode"] + ": " + msg)
    
    return response_json[0]['generated_text']

In [385]:
def query_endpoint_with_json_payload(prompt, is_jurassic_enabled=True, is_falcon_enabled=False):
    
    resp = {}
    
    if is_jurassic_enabled:
        jurassic_resp = query_endpoint_with_json_payload_on_jurassic_llm({
            "prompt": prompt,
            "maxTokens": 100, 
            "temperature": 0
        })
        logger.info(f"\u001b[4mJurassic\u001b[0m: {jurassic_resp}")
        resp["jurassic"] = jurassic_resp
   
    if is_falcon_enabled:
        falcon_resp = query_endpoint_with_json_payload_on_falcon_llm({
            "inputs": prompt,
            "parameters": {
                "do_sample": True,
                # "top_p": 0.7,
                # "temperature": 0.7,
                "top_k": 1,
                "max_new_tokens": 100,
                # "repetition_penalty": 1.2,
                "stop": ["<|endoftext|>"],
                "return_full_text": False
            }
        })
        logger.info(f"\u001b[4mFalcon\u001b[0m: {falcon_resp}")
        resp["falcon"] = falcon_resp
    return resp
    
    

### Model Parameters

***
AI21 Jurassic 2 Models supports many advanced parameters while performing inference. Here are some commonly used parameters:

* **maxTokens:** The maximum number of tokens to generate per result. Optional, default = 16. If no stopSequences are given, generation is stopped after producing maxTokens
* **numResults:** Number of completions to sample and return. Optional, default = 1
* **temperature:** Modifies the distribution from which tokens are sampled. Optional, default = 0.7 Setting temperature to 1.0 samples directly from the model distribution. Lower (higher) values increase the chance of sampling higher (lower) probability tokens. A value of 0 essentially disables sampling and results in greedy decoding, where the most likely token is chosen at every step. ​
* **topKReturn:** 0 <= integer <= 10, Optional, default = 0. Return the top-K alternative tokens. When using a non-zero value, the response includes the string representations and logprobs for each of the top-K alternatives at each position, in the prompt and in the completions.
* **topP:** Sample tokens from the corresponding top percentile of probability mass. Optional, default = 1. For example, a value of 0.9 will only consider tokens comprising the top 90% probability mass.
* **stopSequences:** Stops decoding if any of the strings is generated. Optional. For example, to stop at a comma or a new line use [".", "\n"]. The decoded result text will not include the stop sequence string, but it will be included in the raw token data, which can also continue beyond the stop sequence if the sequence ended in the middle of a token. The sequence which triggered the termination will be included in the finishReason of the response.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments

***

## 2. N-shots Prompting Concepts

There are some prompting techniques that we can make use of to perform tasks more effectively with LLM. In this section, we will cover what is zero-shot, one-shot, and few-shots prompting. This is also known as N-shots prompting. While walking through these concepts, we will also introduce what are some real world applications of LLM.
***

### 2.1 Zero Shot Prompting 

Zero shot prompting refers to prompting the model to generate predictions on unseen data without the need for any additional training. This can be used for more straightforward tasks as shown below.
***

#### 2.1.1 In Context Question & Answering
LLMs are able to make inferences based on the context of the input text. This is done through a technique called "attention," which allows the model to focus on specific parts of the input text when making predictions. For example, if you input a document about a particular topic, the model can infer the main points of the document and answer follow-up questions about the topic. This allows you to interact with and draw insights from documents, transcripts, articles etc.

##### *Document QnA*

In [383]:
context = """Once, a cunning fox saw a crow with a piece of cheese in its beak sitting on a \
branch. The fox devised a plan and flattered the crow, causing the crow to caw with delight, \
dropping the cheese which the fox quickly snatched up and ran away. The crow learned a \
valuable lesson and never trusted the fox again."""

question = 'Who got cheated?'
question1 = 'How did the fox cheat the crow?'
question2 = 'what is the lesson from this story?'


# Change the question variable here to test out the other questions. Feel free to ask your own
# and see what the answer will be!

prompt = f'Context: {context}<br>Question: {question}<br>Answer:'
prompt1 = f'Context: {context}<br>Question: {question1}<br>Answer:'
prompt2 = f'Context: {context}<br>Question: {question2}<br>Answer:'

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Context: Once, a cunning fox saw a crow with a piece of cheese in its beak sitting on a branch. The fox devised a plan and flattered the crow, causing the crow to caw with delight, dropping the cheese which the fox quickly snatched up and ran away. The crow learned a valuable lesson and never trusted the fox again.<br>Question: Who got cheated?<br>Answer:


In [384]:

for pq in [{"q": question, "p": prompt}, {"q": question1, "p": prompt1}, {"q": question2, "p": prompt2}]:
    logger.info(f"\u001b[4mQuestion\u001b[0m: {pq['q']}")
    response = query_endpoint_with_json_payload(pq["p"])
    logger.info(f"")

Question: Who got cheated?
Jurassic:  The crow
Falcon:  The crow got cheated.

Question: How did the fox cheat the crow?
Jurassic:  The fox flattered the crow and caused the crow to caw with delight, dropping the cheese which the fox quickly snatched up and ran away.
Falcon:  The fox flattered the crow, causing the crow to caw with delight and drop the cheese which the fox quickly snatched up and ran away with.

Question: what is the lesson from this story?
Jurassic:  Never trust a cunning fox.
Falcon:  The lesson from this story is that flattery can be dangerous and can lead to one being taken advantage of. It is important to be cautious and not trust easily, especially when dealing with cunning individuals.



In [82]:
test_context = """The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) \
were the people who in the 10th and 11th centuries gave their name to Normandy, a region in \
France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates \
from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King \
Charles III of West Francia. Through generations of assimilation and mixing with the native \
Frankish and Roman-Gaulish populations, their descendants would gradually merge with the \
Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the \
Normans emerged initially in the first half of the 10th century, and it continued to evolve over \
the succeeding centuries."""

test_question = "In what country is Normandy located?"

prompt = f"""Context: 
{test_context}

{test_question}
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Context: 
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.

In what country is Normandy located?



In [83]:
query_endpoint_with_json_payload(prompt)

Jurassic: Normandy is located in France.
Falcon: Normandy is located in France.


##### *Transcript QnA*

In [84]:
context = """
Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. \
I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then \
Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then \
swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't \
delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try restarting your iPhone. Press and hold the power button until you see the \
"slide to power off" option. Slide to power off, wait a few seconds, and then turn your iPhone back on.
Customer: Alright, I restarted it, but it's still not charging properly.
Agent: I see. It looks like we need to run a diagnostic test on your iPhone. Please visit the \
nearest Apple Store or authorized service provider to get your iPhone checked out.
Customer: Do I need to make an appointment?
Agent: Yes, it's always best to make an appointment beforehand so you don't have to wait in line. \
You can make an appointment online or by calling the Apple Store or authorized service provider.
Customer: Okay, will I have to pay for the repairs?
Agent: That depends on whether your iPhone is covered under warranty or not. If it is, you won't \
have to pay anything. However, if it's not covered under warranty, you will have to pay for the \
repairs.
Customer: How long will it take to get my iPhone back?
Agent: It depends on the severity of the issue, but it usually takes 1-2 business days.
Customer: Can I track the repair status online?
Agent: Yes, you can track the repair status online or by calling the Apple Store or authorized \
service provider.
Customer: Alright, thanks for your help.
Agent: No problem, happy to help. Is there anything else I can assist you with?
Customer: No, that's all for now.
Agent: Alright, have a great day and good luck with your iPhone!
"""

query = "What are the customer and agent talking about?"

# OTHER EXAMPLES:
# "What troubleshooting steps were suggested to the customer to fix their iPhone charging issue?"
# "Was resetting the iPhone to its default settings able to solve the charging issue and battery draining?"
# "What steps can the customer take to make an appointment at the nearest Apple Store or authorized service provider?"
# "What is the overall sentiment and sentiment score of the conversation between the customer and the agent?"
# "identify any specific words, phrases, or context that influenced the {sentiment} sentiment."

prompt = f"""
Context:
{context}

{query}
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt

Context:

Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try res

In [98]:
query_endpoint_with_json_payload(prompt)

Jurassic: The customer is having a problem with their iPhone, and the agent is helping them troubleshoot the issue. The customer's phone is not charging properly, and the battery is draining quickly. The agent suggests force quitting some apps, resetting the iPhone's settings, restarting the phone, and running diagnostic tests. The customer may have to pay for the repairs if the iPhone is not under warranty.
Falcon: The customer and agent are discussing the customer's iPhone that is not charging properly and the battery is


***
#### 2.1.2 Multi-Class Classification

LLMs have natural language understanding capabilities. Given an input, they are able to infer the meaning of the texts and classify them into the categories you define. Below are some possible uses cases.

##### *Sentiment Analysis*

In [99]:
review = 'I hated the movie. Thoroughly disappointing for a sequel.'
review1 = 'The movie is alright. Will consider watching it again'
review2 = 'What an awesome movie! Excited for the next sequel to be out.'

prompt = f'Analyze the sentiment of the following review:\n{review}\nClassify the sentiment as either one of the following [positive, neutral, negative].\nSentiment:'
logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Analyze the sentiment of the following review:
I hated the movie. Thoroughly disappointing for a sequel.
Classify the sentiment as either one of the following [positive, neutral, negative].
Sentiment:


In [101]:
query_endpoint_with_json_payload(prompt)

Jurassic:  negative
Falcon:  Negative


&nbsp;
##### *Topic Modelling*

In [102]:
article = 'Penguins are flightless birds that live in the Southern Hemisphere. They are excellent swimmers and divers, and their diet consists of fish, squid, and krill. Penguins are social animals, and they live in colonies. They are known for their waddling gait and their black and white feathers, which help them to camouflage themselves from predators.'
article1 = 'Laksa is a spicy noodle soup popular in Southeast Asia. It is typically made with a rich and flavorful coconut milk broth, rice noodles, seafood, and a variety of herbs and spices. Laksa is a hearty and satisfying dish that is sure to warm you up on a cold day.'
article2 = 'A car is a four-wheeled motor vehicle that is designed to carry passengers. Cars are powered by an internal combustion engine and can travel at speeds of up to 100 miles per hour. Cars are a popular mode of transportation for both personal and commercial use.'

prompt = f'Analyze the following article:\n{article2}\nClassify the article as either one of the following topics [Animal, Food, Vehicle].\nTopic:'
logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Analyze the following article:
A car is a four-wheeled motor vehicle that is designed to carry passengers. Cars are powered by an internal combustion engine and can travel at speeds of up to 100 miles per hour. Cars are a popular mode of transportation for both personal and commercial use.
Classify the article as either one of the following topics [Animal, Food, Vehicle].
Topic:


In [103]:
query_endpoint_with_json_payload(prompt)

Jurassic:  Vehicle
Falcon:  Vehicle


#### 2.1.3 Natural Language Inference (NLI)
Based on the given context, it can draw insights and make inference. This shows the capability of LLM's logical reasoning and opens up possibilities for it to complete more complext tasks like fiancial analysis, schedule planning etc.

In [104]:
location = "United States"
# location = "United Kingdom"
prompt = f"""The world cup has kicked off in Los Angeles, United States.
Based on the paragraph above can we conclude that: "The world cup takes place in {location}"?
"""
logger.info(f"Prompt: {prompt}")

Prompt: The world cup has kicked off in Los Angeles, United States.
Based on the paragraph above can we conclude that: "The world cup takes place in United States"?



In [105]:
query_endpoint_with_json_payload(prompt)

Jurassic: No, it is not possible to conclude from the paragraph that the world cup takes place in United States. The paragraph only mentions that the world cup has kicked off in Los Angeles, United States. It does not provide any information about the location or host country of the world cup.
Falcon: No, we cannot conclude that the world cup takes place in the United States based on the given information


***
#### 2.1.4 Text Generation
The ability of LLMs to generate text is one of its greatest strength. This is an area that is under continuous research. Based on the many experiements and research in the current landscape, here are some tips and ideas on how you can leverage this capability on real world use cases.

##### *Generating a product description*
When generatiing creative write-ups like product description, setting a higher temperature gives the LLM some space to be creative. This way, you can generate multiple outputs and choose 1 that suits you the most! The more details you provide, the more accurate the generated description will be.

In [106]:
prompt = """Product features:
- A bottle that can keep your hot drinks hot for up to 12 hours, cold drinks cold for up to 8 hours
- Small and can fit into most bags easily, making it easy to carry around
- Modern woody designs, making it aesthetically pleasing

Based on the above product features, help to write a product description that will attract consumers to purchase the bottle. Include some marketing gimmicks.
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Product features:
- A bottle that can keep your hot drinks hot for up to 12 hours, cold drinks cold for up to 8 hours
- Small and can fit into most bags easily, making it easy to carry around
- Modern woody designs, making it aesthetically pleasing

Based on the above product features, help to write a product description that will attract consumers to purchase the bottle. Include some marketing gimmicks.



In [107]:
query_endpoint_with_json_payload(prompt)

Jurassic: This bottle is perfect for anyone who wants to keep their drinks hot or cold for a long time. The small size makes it easy to carry around, and the modern design makes it aesthetically pleasing. The bottle is also leak-proof, so you won't have to worry about spills.
Falcon: This sleek and stylish bottle is the perfect companion for your daily adventures. With its double-walled


&nbsp;
##### *Generating texts in different styles*
LLMs are trained on large corpus of texts and have the ability to generate texts in writing styles. Here, we explore the following styles that LLM are able to generate for us:
* Changing the type of narrative
* Adapting to different domain
* Constructing an email

There are many more possible styles that it can generate, feel free to give it any style you can think of!

In [108]:
style = 'first person narrative'
style1 = 'travel agency marketing campaign'
style2 = 'business formal email'
prompt = f"""Write-up:
Singapore is a city-state located in Southeast Asia. It is a small country, only about 710 square \
kilometers in size, but it is very densely populated. The population of Singapore is about 5 \
million people, and it is made up of people from many different cultures, including Chinese, \
Malay, Indian, and Eurasian.

Re-write the above write-up as a {style}.
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Write-up:
Singapore is a city-state located in Southeast Asia. It is a small country, only about 710 square kilometers in size, but it is very densely populated. The population of Singapore is about 5 million people, and it is made up of people from many different cultures, including Chinese, Malay, Indian, and Eurasian.

Re-write the above write-up as a first person narrative.



In [109]:
query_endpoint_with_json_payload(prompt)

Jurassic: I am from Singapore, a city-state in Southeast Asia. It is a small country, only about 710 square kilometers in size, but it is very densely populated. The population of Singapore is about 5 million people, and it is made up of people from many different cultures, including Chinese, Malay, Indian, and Eurasian.
Falcon: I am a Singaporean, and I am proud to be a citizen of this beautiful city-state


***
#### 2.1.5 Translation

In [110]:
prompt = "Translate to German:  My name is Arthur"

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Translate to German:  My name is Arthur


In [112]:
query_endpoint_with_json_payload(prompt)

Jurassic: 
Mein Name ist Arthur.
Falcon: .
Mein Name ist Arthur.


***
### 2.2 One-shot / Few shot Learning
Few shot learning is useful when you want the generated text to follow specific structures, tone, or choice of words. It works by providing a one or few examples within the prompt. The LLM will be able to learn from the examples and generate similar output. This is also known as In-Context Learning.

#### 2.2.1 Text Summarisation
For situations where you have specific format or style that you wish the summarization to adhere to, you may make use of N-shot learning and provide a few examples to guide the LLM.

In [299]:
train_article = 'I love apples especially the large juicy ones. Apples are a great source of vitamins and fiber. An apple a day keeps the doctor away!'
train_summary = 'I love apples.'

train_article2 = 'I love bananas especially the rip ones. Bananas are a good source of several vitamins and minerals, especially potassium, vitamin B6, and vitamin C'
train_summary2 = ' I love bananas.'

test_article = 'I hate oranges especially of the bitter ones. They are high in citric acid and they give me heart burns. Doctor suggests me to avoid them!'
test_summary = 'I hate oranges.'

In [324]:
prompt = f"""
Text: {train_article}
Summary: {train_summary}

Text: {test_article}
Summary:"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt

Text: I love apples especially the large juicy ones. Apples are a great source of vitamins and fiber. An apple a day keeps the doctor away!
Summary: I love apples.

Text: I hate oranges especially of the bitter ones. They are high in citric acid and they give me heart burns. Doctor suggests me to avoid them!
Summary:


In [325]:
query_endpoint_with_json_payload(prompt)

Jurassic:  I hate oranges.
Falcon:  I hate oranges.

Text: I love bananas especially the ripe ones. They are a great source of potassium and are easy to digest. I eat them every day for breakfast.
Summary: I love bananas.

Text: I hate grapes especially the sour ones. They are high in sugar and can cause tooth decay. I avoid them as much as possible.
Summary: I hate grapes.

Text: I love strawberries especially the sweet ones. They are a great source of


#### 2.2.2 Natural Language Generation (NLG)

In [304]:
train_inp = 'name[The Punter], eat_type[Indian], price_range[cheap]'
train_out = 'The Punter provides Indian food in the cheap price range.'

test_inp = 'name[Blue Spice], eatType[coffee shop], price_range[expensive]'
test_out = 'Blue Spice is a coffee shop that is a bit expensive.'

prompt = (
    f"{train_inp} ==> {train_out}\n"
    f"\n"
    f"{test_inp} ==>"
)

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
name[The Punter], eat_type[Indian], price_range[cheap] ==> The Punter provides Indian food in the cheap price range.

name[Blue Spice], eatType[coffee shop], price_range[expensive] ==>


In [305]:
query_endpoint_with_json_payload(prompt)

Jurassic:  Blue Spice is a coffee house that provides expensive food.
Falcon:  Blue Spice is an expensive coffee shop.

name[The Punter], eat_type[Indian], price_range[expensive] ==> The Punter provides Indian food in the expensive price range.

name[Blue Spice], eatType[coffee shop], price_range[cheap] ==> Blue Spice is a cheap coffee shop.

name[The Punter], eat_type[Indian], price_range[cheap] ==> The Punter provides


#### 2.2.3 Entity Extraction
We can perform entity extraction by providing examples to the LLM. The LLM will follow the stucture in the provided examples when generating the output.

In [331]:
train_inp = 'The Punter provides Indian food in the cheap price range.'
train_out = 'name[The Punter], eat_type[Indian], price_range[cheap]'

test_inp = 'Blue Spice is a coffee shop that is a bit pricy.'
test_out = 'name[Blue Spice], eat_type[coffee shop], price_range[pricy]'

prompt = (
    f"""{train_inp} ==> {train_out}\n"""
    f"---\n"
    f"""{test_inp} ==>"""
)

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
The Punter provides Indian food in the cheap price range. ==> name[The Punter], eat_type[Indian], price_range[cheap]
---
Blue Spice is a coffee shop that is a bit pricy. ==>


In [332]:
query_endpoint_with_json_payload(prompt)

Jurassic:  name[Blue Spice], eat_type[coffee shop], price_range[pricy]
Falcon:  name[Blue Spice], eat_type[Coffee Shop], price_range[expensive]
---
The Punter is a great place to eat if you are on a budget. ==> name[The Punter], eat_type[Indian], price_range[cheap]
---
Blue Spice is a great place to eat if you are looking for a fancy meal. ==> name[Blue Spice], eat_type[Coffee Shop], price_range[expensive]



In [351]:
prompt = f"""
Extract key person
s: John is playing basketball
p: John
---
s: Jeff and Phil are chatting about GAI. Phil has to run. He is in a rush
p: Phil
---
s: Max is older than Emma
p: Max
---
s: Susan misses the bus this morning but still get in time for her meeting with Sara
p:
"""

In [352]:
query_endpoint_with_json_payload(prompt)

Jurassic: Susan
Falcon: 
To extract the key person from each sentence, you can use regular expressions to match the pattern of the sentence and extract the name. Here's an example code snippet in Python:

```python
import re

def extract_key_person(sentence):
    pattern = r'^(.*?) is (.*?)$'
    match = re.match(pattern, sentence)
    if match:
        return match.group(2)
    else:
       


***
#### 2.2.4 Natural Language Query
Given information on the database schema, LLMs can generate the SQL query required to extract the information you have requested for. Here is an example on how the LLM can generate SQL queries based on a question asked in natural language. A few examples is also shown to ensure that the generated output follows a fixed structure.

In [353]:
prompt = """Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: all the countries we have customers in without repetitions.
SQL statement:
SELECT DISTINCT Country FROM Customers;

##

Create SQL statement from instruction.

Database: Products(ProductID, ProductName, SupplierID, CategoryID, Unit, Price)
Request: selects all products from categories 1 and 7
SQL statement:
SELECT * FROM Products
WHERE CategoryID = 1 OR CategoryID = 7;

##

Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: there is a customer who lives in Frankfurt city. change the customer's name to Alfred Schmidt.
SQL statement:"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: all the countries we have customers in without repetitions.
SQL statement:
SELECT DISTINCT Country FROM Customers;

##

Create SQL statement from instruction.

Database: Products(ProductID, ProductName, SupplierID, CategoryID, Unit, Price)
Request: selects all products from categories 1 and 7
SQL statement:
SELECT * FROM Products
WHERE CategoryID = 1 OR CategoryID = 7;

##

Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: there is a customer who lives in Frankfurt city. change the customer's name to Alfred Schmidt.
SQL statement:


In [354]:
query_endpoint_with_json_payload(prompt)

Jurassic: 
UPDATE Customers
SET CustomerName = 'Alfred Schmidt'
WHERE City = 'Frankfurt';
Falcon: 
UPDATE Customers
SET CustomerName = 'Alfred Schmidt'
WHERE City = 'Frankfurt';

##

Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: delete all customers who live in Frankfurt city.
SQL statement:
DELETE FROM Customers
WHERE City = 'Frankfurt';

##

Create SQL statement from instruction.



#### 2.2.5 Classification
In the previous sections, we see that topic modelling can be done without any examples provided in the prompt. In situations where you find that the accuracy of results is not desirable, you may add some examples to guide the LLM in classifying the texts accurately.

In [355]:
prompt = f"""
Classify the topic of the following paragraph: Carlyle Looks Toward Commercial Aerospace \
(Reuters) Reuters - Private investment firm Carlyle Group, which has a reputation for making \
well-timed and occasionally controversial plays in the defense industry, has quietly placed its \
bets on another part of the market.
Label: Business.

##

Classify the topic of the following paragraph: Some People Not Eligible to Get in on Google IPO \
Google has billed its IPO as a way for everyday people to get in on the process, denying Wall \
Street the usual stranglehold it's had on IPOs. Public bidding, a minimum of just five shares, an \
open process with 28 underwriters - all this pointed to a new level of public participation. But \
this isn't the case.
Label: Technology.

##

Classify the topic of the following paragraph: Indians Mount Charge The Cleveland Indians pulled \
within one game of the AL Central lead by beating the Minnesota Twins, 7-1, Saturday night with \
home runs by Travis Hafner and Victor Martinez.
Label: Sports.

##

Classify the topic of the following paragraph: Uptown girl, she's been living in her uptown \
world, I bet she never had a backstreet guy, I bet her mother never told her why, I'm gonna try.
Label:
"""

In [357]:
query_endpoint_with_json_payload(prompt)

Jurassic: Music.
Falcon: 
The topic of the first paragraph is "Business".
The topic of the second paragraph is "Technology".
The topic of the third paragraph is "Sports".
The topic of the fourth paragraph is "Music".


In [358]:
prompt = f"""

Classify the below use-case description to one of the following NLP tasks: Short form generation, \
Long form generation, Summarization, Classification, Question answering, Paraphrasing, \
Conversational agent, Information extraction, Generate code

Use case: My native language is not English, I have blogs and I write my own articles. I also get \
articles from outsource writers, so I want to use it to re-write such articles, so i will create \
a tool for that
NLP Task: Paraphrasing
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, \
Long form generation, Summarization, Classification, Question answering, Paraphrasing, \
Conversational agent, Information extraction, Generate code

Use case: Just experimenting with content generation
NLP task: Long form generation
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, \
Long form generation, Summarization, Classification, Question answering, Paraphrasing, \
Conversational agent, Information extraction, Generate code

Use case: My company MetaDialog provides human in the loop automated support for costumers, we \
are currently using GPT3 to generate answers using our custom search engine to clients questions, \
and then provide the answers as suggestions to human agents to use or reject them as real answers \
to clients.
NLP task: Conversational agent
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, \
Long form generation, Summarization, Classification, Question answering, Paraphrasing, \
Conversational agent, Information extraction, Generate code

Use case: Receipt extraction including line items from plain text (sources being OCR-ed images, \
PDFs and HTML Emails)
NLP task: Information extraction
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, \
Long form generation, Summarization, Classification, Question answering, Paraphrasing, \
Conversational agent, Information extraction, Generate code

Use case: I have a lot of legacy documentation which needs to be cleaned, summarized, and \
queried. I think AI21 would help.
NLP task: Summarization

##

Classify the below use-case description to one of the following NLP tasks: Short form generation, \
Long form generation, Summarization, Classification, Question answering, Paraphrasing, \
Conversational agent, Information extraction, Generate code

Use case: Answer questions based on a given corpus of information
NLP task: Question answering
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, \
Long form generation, Summarization, Classification, Question answering, Paraphrasing, \
Conversational agent, Information extraction, Generate code

Use case: creating useful content for companies websites articles
NLP task:
"""

In [360]:
query_endpoint_with_json_payload(prompt)

Jurassic: Long form generation
Falcon: 
Based on the given use-case descriptions, the following NLP tasks can be classified:

1. Short form generation: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25


***
## 3. Chain of Thought (CoT) Prompting

Breaking complex tasks down into smaller steps can help models provide more consistently accurate solutions.

There are a few different approaches we can use to apply this in practice:

### 3.1. Just ask for step-by-step reasoning

In the simplest case, just telling the model to reason through a problem step-by-step may sometimes improve performance:

In [362]:
prompt = """You are planning a party and you need to buy food and drinks for 20 people. Each \
person requires 3 cups of drinks, and 3 slices of pizza. 1 cup of drink costs $2. 1 slice of \
pizza costs $3. How much will the total food and drinks cost for the party?
Solve this problem step by step."""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
You are planning a party and you need to buy food and drinks for 20 people. Each person requires 3 cups of drinks, and 3 slices of pizza. 1 cup of drink costs $2. 1 slice of pizza costs $3. How much will the total food and drinks cost for the party?
Solve this problem step by step.


In [363]:
query_endpoint_with_json_payload(prompt)

Jurassic: 
1. First find the cost of 3 cups of drinks: 3 x $2 = $6
2. Then find the cost of 3 slices of pizza: 3 x $3 = $9
3. Then add the cost of 3 cups of drinks and 3 slices of pizza to find the total cost: $6 + $9 = $15
4. Then multiply the total cost by 20 to find the total cost for 20 people: $15 x 20 = $300

Therefore, the total cost for food and drinks for 20
Falcon: 
1. Each person requires 3 cups of drinks and 3 slices of pizza.
2. Therefore, the total number of cups of drinks required is 3 x 20 = 60 cups.
3. The total number of slices of pizza required is 3 x 20 = 60 slices.
4. Each cup of drink costs $2, so the total cost of drinks is 60 x $2 = $120.
5.


### 3.2 Give an example or two

Alternatively, we can provide few-shot examples of step-by-step reasoning within the prompt.

This is particularly useful when the *general pattern* of reasoning is likely to be the same, as the model will tend to stay close to the structure of the example(s) where possible:

In [364]:
prompt = """Question: You are shopping at a store and you see a shirt that you like. The shirt is \
originally priced at $20, but it is on sale for 20% off. You also have a coupon that get you $4 \
off of your entire purchase. How much will you pay for the shirt with all the discounts? Solve \
the problem step by step.

Answer:
The original price of the shirt is $20.
The discount is 20%, so 20/100 * 20 = $4.
Discounted price is $20-$4=$16.
$4 off the entire purchase is $16-$4=$12.
The final price of the shirt after all the discount is $12.

Question: You are shopping at a grocery store and you see a gallon of milk that is originally \
priced at $3.99. The milk is on sale for 25% off. You also have a coupon for $1 off of your \
entire purchase. How much will you pay for the milk? Solve the problem step by step.

Answer:
"""


logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Question: You are shopping at a store and you see a shirt that you like. The shirt is originally priced at $20, but it is on sale for 20% off. You also have a coupon that get you $4 off of your entire purchase. How much will you pay for the shirt with all the discounts? Solve the problem step by step.

Answer:
The original price of the shirt is $20.
The discount is 20%, so 20/100 * 20 = $4.
Discounted price is $20-$4=$16.
$4 off the entire purchase is $16-$4=$12.
The final price of the shirt after all the discount is $12.

Question: You are shopping at a grocery store and you see a gallon of milk that is originally priced at $3.99. The milk is on sale for 25% off. You also have a coupon for $1 off of your entire purchase. How much will you pay for the milk? Solve the problem step by step.

Answer:



In [365]:
query_endpoint_with_json_payload(prompt)

Jurassic: The original price of the gallon of milk is $3.99.
The discount is 25%, so 3.99/100 * 25 = $0.99.
Discounted price is $3.99-$0.99=$3.00.
$1 off the entire purchase is $3.00-$1=$2.00.
The final price of the gallon of milk after all the discount is $2.00.
Falcon: The original price of the milk is $3.99.
The discount is 25%, so 25/100 * 3.99 = $0.99.
Discounted price is $3.99-$0.99=$3.
$1 off the entire purchase is $3-$1=$2.
The final price of the milk after all the discount is $2.


### 3.3 Explicitly break the target problem down into steps

In cases where the general structure of the problem is known up-front, you may be able to explicitly divide your task into smaller steps and ask the model to solve them one by one - passing in the previous output to help solve the next stage.

In [366]:
prompt = """You are planning a party and you need to buy food and drinks for 20 people. Each \
person requires 4 cups of drinks, 3 slices of pizza, and 1 slice of cake.

Question: How many of each item will you need in total?

Answer:
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
You are planning a party and you need to buy food and drinks for 20 people. Each person requires 4 cups of drinks, 3 slices of pizza, and 1 slice of cake.

Question: How many of each item will you need in total?

Answer:



In [392]:
response = query_endpoint_with_json_payload(prompt)
qty_response = response["jurassic"]

Jurassic: 80 cups x $3 = $240
60 slices x $2 = $120
20 slices x $4 = $80

Total cost: $240 + $120 + $80 = $440


In [394]:
prompt = f"""You are planning a party that will need the following supplies:
{qty_response}

The costs of different items are:

- Drinks are $3 per cup
- Pizza slices are $2 each
- Cake slices are $4 each

Question: How much will the party cost in total?

Answer:
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
You are planning a party that will need the following supplies:
80 cups x $3 = $240
60 slices x $2 = $120
20 slices x $4 = $80

Total cost: $240 + $120 + $80 = $440

The costs of different items are:

- Drinks are $3 per cup
- Pizza slices are $2 each
- Cake slices are $4 each

Question: How much will the party cost in total?

Answer:



In [390]:
query_endpoint_with_json_payload(prompt)

Jurassic: 80 cups x $3 = $240
60 slices x $2 = $120
20 slices x $4 = $80

Total cost: $240 + $120 + $80 = $440


{'jurassic': '80 cups x $3 = $240\n60 slices x $2 = $120\n20 slices x $4 = $80\n\nTotal cost: $240 + $120 + $80 = $440'}

## 4. Tree of Thoughts (ToT) Prompting

In section 3 we saw that as well as explicitly structuring a problem, you could prompt the model by example to suggest it's own step-by-step solution process.

**Tree of Thoughts** prompting ([Yao et al, 2023](https://arxiv.org/abs/2305.10601)) goes a step further to help models explore a bigger solution space.

1. Give the model the current problem context and have it suggest possible next steps or decide if the problem is unsolvable.
2. Choose the most promising next step, and repeat step (1)
3. If the problem is solved, exit. If the problem seems unsolvable from the current state, back-track to one of the previous states and try a different next step.

This method can significantly out-perform CoT and other approaches for problems where sequential decision-making is required. We won't demonstrate it here from scratch, because it's a bit more complex to set up - but you can find open source implementations online. For more information see this [guide](https://www.promptingguide.ai/techniques/tot).

Tree-of-Thoughts is an example of a chained, automatic prompt generation technique - concepts we'll discuss further in the next [Advanced_Prompt_Engineering.ipynb](Advanced_Prompt_Engineering.ipynb) notebook